# Prepare Datasets for Protein Reaction Benchmarking Tasks

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2024-12-14  



## 1. Import packages

In [7]:

import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../')
from config import conf as cfg
from tools import filetool as ftool
from tools import bioFunctionLib as bfl
from tools import rheatool as rheatool
from tqdm import tqdm
from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)
from tkinter import _flatten
import pandas as pd
import numpy as np

FIRST_TIME_RUN = False # For the initial run, please set this flag to True. This will allow the program to download data from UniProt and RHEA, which may take longer depending on your internet speed.

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. 定义处理函数

In [5]:
def prep_pdb(uniprot_id):
    src = f'/hpcfs/fpublic/database/alphafold/predicted_pdbs/AF-{uniprot_id}-F1-model_v4.pdb.gz'
    dst = f'{cfg.DATA_ROOT}structure/pdb/{str(uniprot_id[0:2])}/AF-{uniprot_id}-F1-model_v4.pdb.gz'
    res = ftool.cp_pdb(src, dst)
    # print(res)
    
    return dst, res


def get_pdb_file_from_afdb_web(uniprot_id):
    src = f'https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb'
    dst = f'/hpcfs/fhome/shizhenkun/codebase/RXNRECer/temp/pdb/AF-{uniprot_id}-F1-model_v4.pdb'

    ftool.download_with_wget(src, dst)


def sup_esm_pdb(uniprot_id):
    """ 将ESM计算的PDB 补充到数据库"""
    ESM_OUTPUT_DIR=f"{cfg.RESULTS_DIR}intermediate/esmfold/output/"
    res = False
    src = f'{ESM_OUTPUT_DIR}{uniprot_id}.pdb'
    dst = f'{cfg.DATA_ROOT}structure/pdb/{str(uniprot_id[0:2])}/ESM-{uniprot_id}.pdb'
    
    if os.path.exists(f'{ESM_OUTPUT_DIR}{uniprot_id}.pdb'):
        res = ftool.cp_pdb(src, dst)
    if res:
        return dst
    else:
        return None

## 3. 获取需要计算PDB的所有uniprot_id

In [6]:
ds_train = pd.read_feather(cfg.FILE_DS_TRAIN)
ds_test = pd.read_feather(cfg.FILE_DS_TEST)
data_pdb = pd.concat([ds_train, ds_test], axis=0).reset_index(drop=True)[['uniprot_id', 'seq']]
data_pdb.head(2)

KeyboardInterrupt: 

## 4. 从已知库里（AF2-DB-V4）匹配绝大多数

In [4]:
if FIRST_TIME_RUN:
    if not  os.path.exists(cfg.FILE_DS_PDB_LIST):
        data_pdb[['path_pdb', 'pdb_exist']] = data_pdb.uniprot_id.swifter.apply(lambda x: pd.Series(prep_pdb(x)))
        data_pdb.to_feather(cfg.FILE_DS_PDB_LIST)
    else:
        data_pdb = pd.read_feather(cfg.FILE_DS_PDB_LIST)
else:
    data_pdb = pd.read_feather(cfg.FILE_DS_PDB_LIST)

## 5. 从网络用API补充没有pDB的数据，AF2-DB-ONLINE

In [5]:
if FIRST_TIME_RUN:
    # 指定目录
    directory = "/hpcfs/fhome/shizhenkun/codebase/RXNRECer/temp/pdb/"
    pdb_file_list = ftool.get_pdb_files(directory)

    # 输出文件列表
    print(f"Found {len(pdb_file_list)} .pdb files:")

    #  添加已有数据
    for item in tqdm(pdb_file_list): 
        uniprot_id = item.split('/')[-1].split('-')[1]
        
        try:
            if data_pdb[data_pdb['uniprot_id'] == uniprot_id].pdb_exist.values[0] == False:
                dst = f'{cfg.DATA_ROOT}structure/pdb/{str(uniprot_id[0:2])}/AF-{uniprot_id}-F1-model_v4.pdb'
                res = ftool.cp_pdb(item, dst)
                data_pdb.loc[data_pdb['uniprot_id'] == uniprot_id, 'pdb_exist'] = True
                data_pdb.loc[data_pdb['uniprot_id'] == uniprot_id, 'path_pdb'] = dst
        except Exception as e:
            print(f'{uniprot_id}:{e}')

    # 回写文件
    data_pdb.to_feather(cfg.FILE_DS_PDB_LIST)

## 6. 用ESMFOLD 预测没有的结构

In [6]:
if FIRST_TIME_RUN:
    # 计算需要从官网拿pdb的uniprot_id
    need_wget = data_pdb[data_pdb.pdb_exist==False].reset_index(drop=True)

    # 从 af2 v4 官网下载pdb结构
    tqdm.pandas()
    need_wget.uniprot_id.progress_apply(lambda x: get_pdb_file_from_afdb_web(x))


    # 检查文件是否完整
    data_pdb['pdb_check_ok'] = data_pdb.path_pdb.parallel_apply(lambda x: not ftool.is_file_empty(x))
    need_esm_fold =data_pdb[data_pdb.pdb_check_ok == False].reset_index(drop=True)

    # 制作需要解析PDB的序列文件
    NUM_GPU=32
    chunks = np.array_split(need_esm_fold, NUM_GPU)

    for item in range(len(chunks)):
        bfl.table2fasta(table=chunks[item][['uniprot_id', 'seq']].reset_index(drop=True), file_out=f'{cfg.RESULTS_DIR}intermediate/esmfold/input/chunk{item}.fasta')

    # seq to pdb
    # !sbatch get_esmfold_pdb.sh
    
    
    # 检查文件是否完整
    data_pdb['pdb_check_ok'] = data_pdb.path_pdb.parallel_apply(lambda x: not ftool.is_file_empty(x))
    uniprot_ids_esm_pdb = data_pdb[data_pdb.pdb_check_ok==False].uniprot_id.to_list()

    # 添加已经计算好的PDB
    for uid in tqdm(uniprot_ids_esm_pdb):
        res = sup_esm_pdb(uniprot_id=uid)
        if res!=None:
            data_pdb.loc[data_pdb['uniprot_id'] == uid, 'path_pdb'] = res

    # 回写文件
    data_pdb.to_feather(cfg.FILE_DS_PDB_LIST)

In [7]:
data_pdb

,uniprot_id,seq,path_pdb,pdb_exist,pdb_check_ok
0,Q6GZX4,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
1,Q6GZX3,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
2,Q197F8,MASNTVSAQGGSNRPVRDFSNIQDVAQFLLFDPIWNEQPGSIVPWK...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
3,Q197F7,MYQAINPCPQSWYGSPQLEREIVCKMSGAPHYPNYYPVHPNALGGA...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
4,Q6GZX2,MARPLLGKTSSVRRRLESLSACSIFFFLRKFCQKMASLVFLNSPVY...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
...,...,...,...,...,...
522097,P0DW91,MSGAEEAGGGGPAAGPAGSVPAGVGVGAGAGAGVGVGAGPGAAAGP...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
522098,P0DTL6,MSGAEEAGGGGPAAGPAGSVPAGVGVGVGAGPGAAAGQAAAAALGE...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
522099,P0DW87,MSGAEEAGGGGPAAGPAGAVPAGVGVGAGPGAAAGPAAAALGEAAG...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
522100,P0DW89,MSGAEEAGGGGPAAGPAGAVPAGVGVGVGPGAAAGPAAAALGEAAG...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True


In [11]:
pd.read_feather('http://iof1.tibhpc.net:9000/bucket-brem-web/web_compounds.feather')

,cpd_id,cpd_name,iupac_name,molecular_formula,inchi,inchikey,smiles,synonyms,mol,ref_chebi,ref_pubchem,ref_unichem,num_of_reactions,funcion_group,funcion_group_smarts,canonical_smiles,check
0,CPD:1,3'-sialyllactose,-,C23H39NO19,InChI=1S/C23H39NO19/c1-7(29)24-13-8(30)2-23(22...,-,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)O[C@@](O...,3'-sialyllactose|#35890-38-1|#sialyllactose|#3...,-,-,CID:123914,-,0,[C;D1;H3;!R].[C;D2;H2;!R].[C;D3;H1;!R].[C;D2;H...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D3&H1&!R].[C&D2&H...,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)O[C@@](O...,False
1,CPD:2,1-butanethiol,-,C4H10S,"InChI=1S/C4H10S/c1-2-3-4-5/h5H,2-4H2,1H3",-,CCCCS,1-butanethiol|#butanethiol|#butane-1-thiol|#10...,-,-,CID:8012,-,1,[C;D1;H3;!R].[C;D2;H2;!R].[SX2;H1;!R],[C&D1&H3&!R].[C&D2&H2&!R].[S&X2&H1&!R],CCCCS,False
2,CPD:3,N-(5-aminopentyl)acetamide,-,C7H16N2O,"InChI=1S/C7H16N2O/c1-7(10)9-6-4-2-3-5-8/h2-6,8...",-,CC(=O)NCCCCCN,n-(5-aminopentyl)acetamide|#32343-73-0|#monoac...,-,-,CID:189087,-,3,[C;D1;H3;!R].[C;D2;H2;!R].[C;D3;H0;!$(CO)](=[O...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D3&H0&!$(CO)]=[O&...,CC(=O)NCCCCCN,False
3,CPD:4,D-alt,-,C6H12O6,InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2...,-,OC[C@H]1OC(O)[C@@H](O)[C@H](O)[C@@H]1O,d-alt|#d-altropyranose|#d-altropyranoside|#d-a...,-,-,CID:441032,-,2,[C;D2;H2;!R].[C;D3;H1;R].[C][O;D2;H0;!$(OC=O);...,[C&D2&H2&!R].[C&D3&H1&R].C[O&D2&H0&!$(OC=O)&!$...,OC[C@H]1OC(O)[C@@H](O)[C@H](O)[C@@H]1O,False
4,CPD:5,A very-long-chain 3-oxoacyl-coa,-,C24H33N7O18P3SR,,-,*C(=O)CC(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(C)(C)CO...,a very-long-chain 3-oxoacyl-coa,-,CHEBI:90725,-,-,0,[C;D1;H3;!R].[C;D2;H2;!R].[C;D3;H1;!R].[C;D4;H...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D3&H1&!R].[C&D4&H...,*C(=O)CC(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(C)(C)CO...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80265,CPD:80266,Apcp,-,-,InChI=1S/C19H26N8O14P2/c20-9-1-2-26(19(31)25-9...,-,-,-,-,-,-,-,2,-,-,Nc1ccn([C@@H]2O[C@H](COP(=O)(O)O[C@H]3[C@@H](O...,False
80266,CPD:80267,-,-,-,InChI=1S/C29H40N13O27P5/c30-12-1-2-40(29(48)37...,-,-,-,-,-,-,-,1,-,-,Nc1ccn([C@@H]2O[C@H](COP(=O)(O)O[C@H]3[C@@H](O...,False
80267,CPD:80268,-,-,-,InChI=1S/C29H40N13O27P5/c30-12-1-2-40(29(48)37...,-,-,-,-,-,-,-,1,-,-,N=c1[nH]c(=O)c2ncn([C@@H]3O[C@H](COP(=O)(O)OP(...,False
80268,CPD:80269,D-myo-inositol,-,-,InChI=1S/C6H12O6/c7-1-2(8)4(10)6(12)5(11)3(1)9...,-,-,-,-,-,-,-,1,-,-,OC1[C@H](O)[C@@H](O)[C@@H](O)[C@H](O)[C@@H]1O,True


In [9]:
pd.read_feather('http://iof1.tibhpc.net:9000/bucket-brem-web/web_reactions.feather')

,reaction_id,equation_string,equation_string_new,equation,equation_smiles,reversible,natural,ec,organism,protein_refs,...,num_of_enzymes,rxn_atom_mapped_enzymemap,rxn_center,lb_rxn_center_s1,rxn_atom_mapped_enzymemap_single,rxn_atom_mapped_rxnmapper,lb_rxn_center_s1_enzymemap,rxn_pic,rxn_atom_mapped_enzymemap_rdt,num_of_reactions
0,RXN:1,valeraldehyde + NADH = 1-pentanol + NAD+,valeraldehyde + NADH + h+ = 1-pentanol + NAD+,CPD:9339 + CPD:1059 + CPD:53737>>CPD:3673 + CP...,CCCCC=O.NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP...,{r},False,"1.1.1.21,1.1.1.1,1.1.1.202","Rhodococcus ruber , Bos taurus, Euonymus japon...","A3MVR8 , Q59477",...,2,[CH3:1][CH2:2][CH2:3][CH2:4][CH:5]=[O:6].[H+]....,[C:5].[O:6].[C:10].[C:11].[N:12].[C:48].[C:49]...,COCCNCCC>>COCCNCCC,[CH3:1][CH2:2][CH2:3][CH2:4][CH:5]=[O:6].[H+]....,[CH3:1][CH2:2][CH2:3][CH2:4][CH:5]=[O:6].[NH2:...,[C:6].[O:8] + [C:1].[C:2].[N:3].[C:4].[C:5].[C...,[['pic/reaction/0f62ab64d2071a23759836fc022252...,[O:1]=[C:2]([OH:3])[C:4](=[O:5])[CH2:6][CH2:7]...,NaN
1,RXN:2,pyridinecarboxaldehyde + NAD(P)H = pyridylmeth...,NADH + 2-pyridinecarboxaldehyde + h+ = NAD+ + ...,CPD:1059 + CPD:4054 + CPD:53737>>CPD:53738 + C...,NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)O...,{undefined},False,1.1.1.21,"Homo sapiens, Euonymus japonicus",,...,0,[H+].[NH2:1][C:2](=[O:3])[C:4]1=[CH:5][N:6]([C...,[C:4].[C:5].[N:6].[C:42].[C:43].[C:44].[O:45]....,CCNCCCOC>>CCNCCCOC,[H+].[NH2:1][C:2](=[O:3])[C:4]1=[CH:5][N:6]([C...,[NH2:1][C:2](=[O:3])[C:4]1=[CH:44][N:8]([C@@H:...,[C:1].[C:7].[N:8].[C:2].[C:3].[C:4] + [O:5].[C...,[['pic/reaction/366beba58e60902bd4e2070e4880ae...,[O:1]=[C:2]([OH:3])[C@@H:4]([NH2:5])[CH2:6][CH...,NaN
2,RXN:3,S-adenosyl-L-methionine + L-Lys = S-adenosyl-L...,S-adenosyl-L-methionine + l-lysine = N(2)-meth...,CPD:19654 + CPD:2643>>CPD:48378 + CPD:2891 + C...,C[S+](CC[C@H](N)C(=O)O)C[C@H]1O[C@@H](n2cnc3c(...,{undefined},False,"2.1.1.301,2.1.1.127","Pisum sativum , Streptomyces sp.",Q43088,...,1,[CH3:1][S+:2]([CH2:3][CH2:4][C@H:5]([NH2:6])[C...,[C:1].[S:2].[N:34]>>[C:1].[N:34].[S:2],CSN>>CNS,[CH3:1][S+:2]([CH2:3][CH2:4][C@H:5]([NH2:6])[C...,[CH3:1][S+:26]([CH2:25][C@H:24]1[O:23][C@@H:22...,[C:2].[S:3] + [N:4]>>[C:2].[N:4] + [H:1] + [S:3],[['pic/reaction/63fb9ff0ebae9695411c0f3c8f177f...,[O:1]=[C:2]([OH:3])[C@@H:4]([NH2:5])[CH2:6][CH...,NaN
3,RXN:4,S-adenosyl-L-methionine + methyl-L-Lys = S-ade...,N(2)-methyl-L-lysine + S-adenosyl-L-methionine...,CPD:48378 + CPD:19654>>CPD:57088 + CPD:2891 + ...,CN[C@@H](CCCCN)C(=O)O.C[S+](CC[C@H](N)C(=O)O)C...,{undefined},False,2.1.1.127,Pisum sativum,Q43088,...,1,[CH3:12][S+:13]([CH2:14][CH2:15][C@H:16]([NH2:...,[C:12].[S:13].[N:8]>>[N:8].[C:12].[S:13],CSN>>NCS,[CH3:12][S+:13]([CH2:14][CH2:15][C@H:16]([NH2:...,[NH2:2][CH2:3][CH2:4][CH2:5][CH2:6][C@H:7]([NH...,[C:2].[S:3] + [N:4]>>[N:4].[C:2] + [H:1] + [S:3],[['pic/reaction/629a18db4bd177b087e8d3d6624c7e...,[O:1]=[C:2]([OH:3])[C@@H:4]([NH2:5])[CH2:6][CH...,NaN
4,RXN:5,S-adenosyl-L-methionine + (S)-norlaudanosoline...,S-adenosyl-L-methionine + (S)-norlaudanosoline...,CPD:19654 + CPD:20620>>CPD:21770 + CPD:2891 + ...,C[S+](CC[C@H](N)C(=O)O)C[C@H]1O[C@@H](n2cnc3c(...,{undefined},True,2.1.1.128,"Coptis japonica, Papaver somniferum, Eschschol...",,...,0,[CH3:1][S+:2]([CH2:3][CH2:4][C@H:5]([NH2:6])[C...,[C:1].[S:2].[O:41]>>[C:1].[O:41].[S:2],CSO>>COS,[CH3:1][S+:2]([CH2:3][CH2:4][C@H:5]([NH2:6])[C...,[CH3:1][S+:37]([CH2:36][C@H:35]1[O:34][C@@H:33...,[C:2].[S:3] + [O:4]>>[C:2].[O:4] + [H:1] + [S:3],[['pic/reaction/63fb9ff0ebae9695411c0f3c8f177f...,[O:1]=[C:2]([OH:3])[C@@H:4]([NH2:5])[CH2:6][CH...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30217,RXN:54632,S-adenosyl-L-methionine + 10-hydroxydihydrosan...,10-hydroxydihydrosanguinarine + S-adenosyl-L-m...,CPD:24921 + CPD:19654>>CPD:7427 + CPD:2891 + C...,CN1Cc2c3c(cc(O)c2-c2ccc4cc5c(cc4c21)OCO5)OCO3....,{undefined},True,2.1.1.119,Eschscholzia californica,,...,0,[CH3:1][N:2]1[CH2:3][c:4]2[c:5]3[c:6]([cH:7][c...,[O:9].[C:27].[